In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

In [2]:
cleaned_file = 'otodomScrapRAWData_cleaned.csv'
df = pd.read_csv(cleaned_file)
print(df.shape)
df.head(6)

(17010, 12)


,price,latitude,longitude,surface_area,num_of_room,floor,num_floors_in_building,finish_condition,form_of_property,balcony_garden_terrace,parking_space,heating
0,1109400.0,52.196401,20.952040,73.06,4,1.0,7.0,for finishing,full ownership,Balcony,garage/parking space,NaN
1,NaN,52.112868,20.973348,47.87,2,0.0,3.0,for finishing,full ownership,"Terrace, Garden",NaN,NaN
2,1255000.0,52.233130,21.019000,42.00,3,4.0,6.0,for living,full ownership,Balcony,NaN,municipal
3,980000.0,52.272250,20.928295,60.00,2,0.0,7.0,for living,full ownership,Garden,garage/parking space,municipal
4,880000.0,52.238722,20.961506,49.90,2,7.0,7.0,for living,NaN,NaN,NaN,municipal
5,NaN,52.164608,20.799947,130.46,4,2.0,4.0,for finishing,full ownership,Balcony,garage/parking space,NaN


The `balcony_garden_terrace` column has multiple values so an regular `OneHotEncoder` will not work here.

Therefore, we will use a specialized `MultiLabelBinarizer` for this purpose </p>

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer

# Step 0: Becuase we have some NaN values, it will throw AttributeError: 'float' object has no attribute 'split'
df['balcony_garden_terrace'] = df['balcony_garden_terrace'].fillna("Placeholder")

# Step 1: Split the strings into lists
df['balcony_garden_terrace'] = df['balcony_garden_terrace'].apply(lambda x: x.split(', '))

# Step 2: Apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
one_hot_encoded = mlb.fit_transform(df['balcony_garden_terrace'])

# Create a DataFrame with the one-hot encoded columns
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=mlb.classes_, index=df.index)

# Step 3: Combine the one-hot encoded columns with the original DataFrame
df = pd.concat([df, one_hot_encoded_df], axis=1)

# Drop the original 'balcony_garden_terrace' column
df = df.drop(["balcony_garden_terrace", "Placeholder"], axis=1)

# Display the resulting DataFrame
df.head(6)

,price,latitude,longitude,surface_area,num_of_room,floor,num_floors_in_building,finish_condition,form_of_property,parking_space,heating,Balcony,Garden,Terrace
0,1109400.0,52.196401,20.952040,73.06,4,1.0,7.0,for finishing,full ownership,garage/parking space,NaN,1,0,0
1,NaN,52.112868,20.973348,47.87,2,0.0,3.0,for finishing,full ownership,NaN,NaN,0,1,1
2,1255000.0,52.233130,21.019000,42.00,3,4.0,6.0,for living,full ownership,NaN,municipal,1,0,0
3,980000.0,52.272250,20.928295,60.00,2,0.0,7.0,for living,full ownership,garage/parking space,municipal,0,1,0
4,880000.0,52.238722,20.961506,49.90,2,7.0,7.0,for living,NaN,NaN,municipal,0,0,0
5,NaN,52.164608,20.799947,130.46,4,2.0,4.0,for finishing,full ownership,garage/parking space,NaN,1,0,0


In [4]:
print(df['Balcony'].dtype)

int32


In [5]:
df['parking_space'] = df['parking_space'].apply(lambda x: 1 if x == 'garage/parking space' else 0).astype('int8')

In [6]:
df = pd.concat([df, pd.get_dummies(df['form_of_property'])], axis=1)
df = df.drop(['form_of_property'], axis=1)

In [7]:
df.head(5)

,price,latitude,longitude,surface_area,num_of_room,floor,num_floors_in_building,finish_condition,parking_space,heating,Balcony,Garden,Terrace,cooperative ownership right to the premises,full ownership,perpetual usufruct / lease 29,share
0,1109400.0,52.196401,20.952040,73.06,4,1.0,7.0,for finishing,1,NaN,1,0,0,False,True,False,False
1,NaN,52.112868,20.973348,47.87,2,0.0,3.0,for finishing,0,NaN,0,1,1,False,True,False,False
2,1255000.0,52.233130,21.019000,42.00,3,4.0,6.0,for living,0,municipal,1,0,0,False,True,False,False
3,980000.0,52.272250,20.928295,60.00,2,0.0,7.0,for living,1,municipal,0,1,0,False,True,False,False
4,880000.0,52.238722,20.961506,49.90,2,7.0,7.0,for living,0,municipal,0,0,0,False,False,False,False


In [8]:
df = pd.concat([df, pd.get_dummies(df['heating'])], axis = 1)
df = df.drop(['heating'], axis=1)

In [9]:
df.head(6)

,price,latitude,longitude,surface_area,num_of_room,floor,num_floors_in_building,finish_condition,parking_space,Balcony,...,cooperative ownership right to the premises,full ownership,perpetual usufruct / lease 29,share,boiler room,electric,gas,municipal,other,piece kaflowe
0,1109400.0,52.196401,20.952040,73.06,4,1.0,7.0,for finishing,1,1,...,False,True,False,False,False,False,False,False,False,False
1,NaN,52.112868,20.973348,47.87,2,0.0,3.0,for finishing,0,0,...,False,True,False,False,False,False,False,False,False,False
2,1255000.0,52.233130,21.019000,42.00,3,4.0,6.0,for living,0,1,...,False,True,False,False,False,False,False,True,False,False
3,980000.0,52.272250,20.928295,60.00,2,0.0,7.0,for living,1,0,...,False,True,False,False,False,False,False,True,False,False
4,880000.0,52.238722,20.961506,49.90,2,7.0,7.0,for living,0,0,...,False,False,False,False,False,False,False,True,False,False
5,NaN,52.164608,20.799947,130.46,4,2.0,4.0,for finishing,1,1,...,False,True,False,False,False,False,False,False,False,False


In [10]:
# df['finish_condition'] = df['finish_condition'].apply(lambda x:
#                                                       1 if x == 'for living'
#                                                       else 0.8 if x == 'for renovation'
#                                                       else 0.5 if pd.isnull(x)
#                                                       else 0 if x == 'for finishing'
#                                                       else 0.5)
# 'for living'
# 'for finishing'
# 'for renovation'
#  NaN

In [11]:
from sklearn.model_selection import train_test_split

# Split the DataFrame into two parts based on whether 'price' is missing
# df_without_price = df[df['price'].isna()]
df_without_price = df[pd.isnull(df['price'])]

df = df[df['price'].notna()]

print(df_without_price.shape)
print(df.shape)

(2205, 22)
(14805, 22)


In [12]:
df.sample(5)

,price,latitude,longitude,surface_area,num_of_room,floor,num_floors_in_building,finish_condition,parking_space,Balcony,...,cooperative ownership right to the premises,full ownership,perpetual usufruct / lease 29,share,boiler room,electric,gas,municipal,other,piece kaflowe
12467,670000.0,52.198067,20.958220,37.68,2,6.0,6.0,for renovation,1,1,...,False,True,False,False,False,False,False,True,False,False
13844,950000.0,52.159670,21.033060,82.00,4,2.0,4.0,for renovation,0,0,...,False,False,False,False,False,False,False,True,False,False
235,436962.0,52.080951,21.130732,38.00,2,4.0,5.0,for renovation,0,0,...,False,False,False,False,True,False,False,False,False,False
6768,693840.0,52.259902,21.040038,28.91,1,4.0,6.0,NaN,0,0,...,False,True,False,False,False,False,False,False,False,False
9717,768000.0,52.321722,20.963644,53.70,3,9.0,9.0,for living,1,0,...,False,True,False,False,False,False,False,True,False,False


In [13]:
df.columns

Index(['price', 'latitude', 'longitude', 'surface_area', 'num_of_room',
       'floor', 'num_floors_in_building', 'finish_condition', 'parking_space',
       'Balcony', 'Garden', 'Terrace',
       'cooperative ownership right to the premises', 'full ownership',
       'perpetual usufruct / lease 29', 'share', 'boiler room', 'electric',
       'gas', 'municipal', 'other', 'piece kaflowe'],
      dtype='object')


## Hypothesis Testing

In [14]:
price_living = df[df['finish_condition'] == 'for living']['price']
price_finishing = df[df['finish_condition'] == 'for finishing']['price']
print(price_living.shape)
print(price_finishing.shape)
print(price_living.dtype)
print(price_finishing.dtype)

(7455,)
(3860,)
float64
float64


In [15]:
import scipy as sp
SIGNIFICANCE_LEVEL = 0.05

t_stat, p_value = sp.stats.ttest_ind(price_living, price_finishing)
print(t_stat, p_value)

rejecting = p_value < SIGNIFICANCE_LEVEL
print("NULL Hypth. H0: A will-be-ready-to-live house is cheaper than a ready-to-live house.")
print(("" if rejecting else "Not ") + "possible to reject the null hypothesis")

15.651347842069383 1.2150577546499431e-54
NULL Hypth. H0: A will-be-ready-to-live house is cheaper than a ready-to-live house.
possible to reject the null hypothesis


## RandomForestRegressor to predict price

In [16]:
df.sample(5)

,price,latitude,longitude,surface_area,num_of_room,floor,num_floors_in_building,finish_condition,parking_space,Balcony,...,cooperative ownership right to the premises,full ownership,perpetual usufruct / lease 29,share,boiler room,electric,gas,municipal,other,piece kaflowe
7728,528000.0,52.378040,20.920890,38.85,2,3.0,3.0,for living,1,1,...,False,True,False,False,False,False,False,False,False,False
1812,670000.0,52.192299,20.903424,44.70,2,1.0,6.0,for living,1,1,...,False,True,False,False,False,False,False,True,False,False
13448,1290000.0,52.160350,20.994790,64.95,3,3.0,6.0,for living,1,1,...,False,False,False,False,False,False,False,True,False,False
12947,620000.0,52.299806,21.042123,48.00,2,9.0,10.0,NaN,0,1,...,False,True,False,False,False,False,False,True,False,False
12131,770000.0,52.235840,20.940380,48.20,3,1.0,4.0,for renovation,0,1,...,False,False,False,False,False,False,False,True,False,False


In [17]:
df.isnull().sum()

price                                             0
latitude                                          0
longitude                                         0
surface_area                                      0
num_of_room                                       0
floor                                           344
num_floors_in_building                          779
finish_condition                               2389
parking_space                                     0
Balcony                                           0
Garden                                            0
Terrace                                           0
cooperative ownership right to the premises       0
full ownership                                    0
perpetual usufruct / lease 29                     0
share                                             0
boiler room                                       0
electric                                          0
gas                                               0
municipal   

In [18]:
print(df.shape)
df = df.dropna(subset=['floor', 'num_floors_in_building'])
print(df.shape)

(14805, 22)
(14021, 22)


In [19]:
X = df.drop(['price', 'finish_condition'], axis=1)
y = df['price']
X.isnull().sum()

latitude                                       0
longitude                                      0
surface_area                                   0
num_of_room                                    0
floor                                          0
num_floors_in_building                         0
parking_space                                  0
Balcony                                        0
Garden                                         0
Terrace                                        0
cooperative ownership right to the premises    0
full ownership                                 0
perpetual usufruct / lease 29                  0
share                                          0
boiler room                                    0
electric                                       0
gas                                            0
municipal                                      0
other                                          0
piece kaflowe                                  0
dtype: int64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

In [21]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(random_state=24)
forest.fit(X_train, y_train)

forest_score = forest.score(X_test, y_test)

print("RandomForestRegressor Score: ", forest_score)

RandomForestRegressor Score:  0.893947679410712


## Mean Absolute Error (MAE)
## Mean Squared Error (MSE)
## R-Squared (R2)

In [25]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions
price_predict = forest.predict(X_test)

# Calculate metrics
mae = mean_absolute_error(y_test, price_predict)
mse = mean_squared_error(y_test, price_predict)
r2 = r2_score(y_test, price_predict)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Absolute Error: 92080.33564375936
Mean Squared Error: 24661109760.78654
R-squared: 0.893947679410712
